In [8]:
!python setup.py build_ext --inplace

(worker pid=5845) trial_num 7
(worker pid=5845)   action
(worker pid=5845) 0   left
(worker pid=5845) chosen_action: stop
(worker pid=5845) trial_num 7
(worker pid=5845)   action
(worker pid=5845) 0   left
(worker pid=5845) chosen_action: stop
running build_ext
running build_ext


In [9]:
import network_simulation_freeze_stop_amplitudes as net_sims
import popconstruct as pops
import plotting_helper_functions as plt_help
import pandas as pd
import os
import time
import pdb

import pylab as pl
import pickle
import numpy as np
import pandas as pd
import pingouin
import sklearn 

def saveresults_vars(variable, prefix):
    pickle.dump(variable, open(prefix, 'wb'))
    
def loadresults_vars(prefix):
    return pickle.load(open(prefix, "rb"))

data_dir = "./Data/stop-params_freeze-analysis/amplitudes/iSPN+GPeA_10threads/"
figure_dir = "./Figures/"

In [10]:
experimentchoice = 'stopsignal'
number_of_choices = 1
#pl_creat.choose_pipeline(experimentchoice)

In [11]:
environment = {
    
    'experimentchoice': experimentchoice,
    'inter_trial_interval': None,
    'thalamic_threshold': 30.,
    'movement_time': ['constant', 300], #['constant', 300], #default sampled from N(250,1.5), ["constant",250], ["mean",250]
    'choice_timeout': 300, #default 1000
    
    'params': None, #neuron parameters (init_params.py)
    'pops': None,   #population parameters (init_params.py)
    'receps' : None, #receptor parameters (init_params.py)
    'base' : None, #baseline stimulation parameters (init_params.py)
    'dpmns' : None, #dopamine related parameters (init_params.py)
    'd1' : None, #D1-MSNs population related parameters (init_params.py)
    'd2' : None, #D2-MSNs population related parameters (init_params.py)
    'channels' : pd.DataFrame([["left"]], columns=['action']), #action channels related parameters (init_params.py)
    'number_of_choices': number_of_choices,
    #'actionchannels' : pd.DataFrame([[1],[2]], columns=['action']), #labels for the actions (init_params.py)
    #'actionchannels' : pd.DataFrame([["left"],["right"]], columns=['action']), #labels for the actions (init_params.py)
    'newpathways' : None, #connectivity parameters (popconstruct.py)
    'Q_support_params': None, #initialization of Q-values update (qvalues.py) 
    'Q_df_set': pd.DataFrame([[0.5]],columns=["left"]), #pd.DataFrame([[0.3,0.7]],columns=['left','right']), #initialized Q-values df (qvalues.py)  
    'n_trials': 30, #number of trials (generateepochs.py)
    'volatility': [None,"exact"], #frequency of changepoints (generateepochs.py)
    'conflict': (1.0), #probability of the preferred choice (generateepochs.py)
    'reward_mu': 1, #mean for the magnitude of the reward (generateepochs.py)
    'reward_std': 0.1, #std for the magnitude of the reward (generateepochs.py)
    'maxstim': 1.15, # amplitude of the cortical input over base line #0.85
    'sustainedfraction': 0.75,
    
    #Stop 1
    'stop_signal_present': True,
    'stop_signal_probability': 1., #probability of trials that will get the stop signal / list of trial numbers
    'stop_signal_amplitude': 0.4, # amplitude of the stop signal over base line
    'stop_signal_onset': 70., #in ms #antes 200
    'stop_signal_duration' : 145., 
    'stop_signal_channel': "all", #"all" (all channels are given the stop signal) 
                                #/"any" (channel given the stop signal is chosen randomly)
                                # [list of channels] == subset of channels given the stop signa
    'stop_signal_population':["STNE"],
    
    #Stop 2
    'stop_2_signal_present': True,
    'stop_2_signal_probability': 1., #probability of trials that will get the stop signal / list of trial numbers
    'stop_2_signal_amplitude': 0.4, # amplitude of the stop signal over base line
    'stop_2_signal_onset': 70., #in ms #antes 200
    'stop_2_signal_duration' : 145.,
    'stop_2_signal_channel': "all", #"all" (all channels are given the stop signal) 
                                #/"any" (channel given the stop signal is chosen randomly)
                                # [list of channels] == subset of channels given the stop signal
    'stop_2_signal_population':["GPeA"],                           
    
    
    'decision_channel' : 'all', #'all'(all channels)
                              # '0' : left channel considered 
                              # '1' : right channel considered
    
    'record_variables': [], #["weight","optogenetic_input", "stop_input_1", "stop_input_2"],
    
    #Opto
    'opt_signal_present': True,
    'opt_signal_probability': 1., #[0,1,2,3,4], # probability of trials that will get the optogenetic signal / list of trial numbers
    'opt_signal_amplitude': 0.4, # ampitude of the stop signal over base line
    'opt_signal_onset': 70., # in ms
    'opt_signal_duration': 145.,
    'opt_signal_channel': "all", # "all" (all channels are given the stop signal) 
    'opt_signal_population':["D2STR"],    
}

In [12]:
solutions_amplitude_GPeA = np.arange(0.2, 0.9, 0.1)
solutions_amplitude_iSPN = np.arange(0.2, 0.9, 0.1)
print(solutions_amplitude_GPeA)
print(solutions_amplitude_iSPN)

[0.2 0.3 0.4 0.5 0.6 0.7 0.8]
[0.2 0.3 0.4 0.5 0.6 0.7 0.8]


In [13]:
offset = 0

In [14]:
for i in np.arange(len(solutions_amplitude_GPeA)):
    if i < offset:
        continue
    print("i:",i)
                 
    amplitude_GPeA = solutions_amplitude_GPeA[i]
    
    for j in np.arange(len(solutions_amplitude_iSPN)):
        
        print("j:", j) 
        
        seed_1 = i
        seed_2 = j
        
        amplitude_iSPN = solutions_amplitude_iSPN[j]
        
        if os.path.exists(data_dir+'network_data_onset'+str(np.round(amplitude_GPeA, 1))+'_iSPN_'+str(np.round(amplitude_iSPN, 1))):
            continue
        
        new_env = environment.copy()
        new_env['stop_2_signal_amplitude'] = amplitude_GPeA
        new_env['opt_signal_amplitude'] = amplitude_iSPN
        
        print('Amplitude GPeA:', np.round(amplitude_GPeA, 1))
        print('Amplitude iSPN:', np.round(amplitude_iSPN, 1))

        results = net_sims.run_simulation(seed_1, seed_2, amplitude_GPeA, amplitude_iSPN, new_env)

i: 0
j: 0
j: 1
j: 2
j: 3
j: 4
j: 5
j: 6
i: 1
j: 0
j: 1
j: 2
j: 3
j: 4
j: 5
j: 6
i: 2
j: 0
j: 1
j: 2
j: 3
j: 4
j: 5
j: 6
i: 3
j: 0
j: 1
j: 2
j: 3
j: 4
j: 5
j: 6
i: 4
j: 0
j: 1
j: 2
j: 3
j: 4
j: 5
j: 6
i: 5
j: 0
j: 1
j: 2
j: 3
j: 4
j: 5
j: 6
i: 6
j: 0
j: 1
j: 2
Amplitude GPeA: 0.8
Amplitude iSPN: 0.4
{'experimentchoice': 'stopsignal', 'inter_trial_interval': None, 'thalamic_threshold': 30.0, 'movement_time': ['constant', 300], 'choice_timeout': 300, 'params': None, 'pops': None, 'receps': None, 'base': None, 'dpmns': None, 'd1': None, 'd2': None, 'channels':   action
0   left, 'number_of_choices': 1, 'newpathways': None, 'Q_support_params': None, 'Q_df_set':    left
0   0.5, 'n_trials': 30, 'volatility': [None, 'exact'], 'conflict': 1.0, 'reward_mu': 1, 'reward_std': 0.1, 'maxstim': 1.15, 'sustainedfraction': 0.75, 'stop_signal_present': False, 'stop_signal_probability': 1.0, 'stop_signal_amplitude': 0.4, 'stop_signal_onset': 30.0, 'stop_signal_duration': 150.0, 'stop_signal_channel': 'a

(worker pid=5817) [ 0 18 28 17  5 26  7 14 19 21 22 29  2 12 24  3 23  9  6 11  8 13 10 15
(worker pid=5817)  25 16 27  4 20  1]
(worker pid=5823) [19  3  4 20 26 25 10  8  1  0 27 29 16  5 14 13  9 22 18 23  6 17 11 24
(worker pid=5823)   7 28 15  2 21 12]
(worker pid=5823) [19  3  4 20 26 25 10  8  1  0 27 29 16  5 14 13  9 22 18 23  6 17 11 24
(worker pid=5823)   7 28 15  2 21 12]
(worker pid=5823) [23  1 15  5 21 27 28  2 18  6 10 16  9  4 12 11 22 17 13  7 25 26  0 14
(worker pid=5823)  29  8 19  3 20 24]
(worker pid=5823) [23  1 15  5 21 27 28  2 18  6 10 16  9  4 12 11 22 17 13  7 25 26  0 14
(worker pid=5823)  29  8 19  3 20 24]
(worker pid=5823) [ 6 22 23 24 27 26  1 18 12 20 19 16 17 28  2  9  3 14 10 29  7  4 11 21
(worker pid=5823)  25 15  8  0  5 13]
(worker pid=5823) [ 6 22 23 24 27 26  1 18 12 20 19 16 17 28  2  9  3 14 10 29  7  4 11 21
(worker pid=5823)  25 15  8  0  5 13]
(worker pid=5817) channels: one
(worker pid=5817) scaling_conn 1
(worker pid=5817) scaling_wts 2.

(worker pid=5822) [ 5  7  9  1 25 27  4 23 29 15 17 12 20 26 21 11 18  3  2 13  8 24 10 14
(worker pid=5822)  19  6 28 22  0 16]
SpawnThreadManagers <backend.Pipeline object at 0x7f806079fe80>
(worker pid=5822) [ 5  7  9  1 25 27  4 23 29 15 17 12 20 26 21 11 18  3  2 13  8 24 10 14
(worker pid=5822)  19  6 28 22  0 16]
SpawnThreadManagers <backend.Pipeline object at 0x7f80607a0640>
(worker pid=5817) [ 8 28 21 22  5 11  9 27  6 15  0  4 24 23 12  2 26 29  1 20 10  3 14 25SpawnThreadManagers <backend.Pipeline object at 0x7f80607a0640>

(worker pid=5817)   7 17 19 18 13 16]
(worker pid=5817) [ 8 28 21 22  5 11  9 27  6 15  0  4 24 23 12  2 26 29  1 20 10  3 14 25
(worker pid=5817)   7 17 19 18 13 16]
(worker pid=5841) channels: one
(worker pid=5841) scaling_conn 1
(worker pid=5841) scaling_wts 2.0
(worker pid=5841) channels: one
(worker pid=5841) scaling_conn 1
(worker pid=5841) scaling_wts 2.0
(worker pid=5841) channels: one
(worker pid=5841) scaling_conn 1
(worker pid=5841) scaling_wts

(worker pid=5843) stop_2 stim started
(worker pid=5843) opt stim started
(worker pid=5843) stop_2 stim started
(worker pid=5843) opt stim started
(worker pid=5846) stop_2 stim started
(worker pid=5846) opt stim started
(worker pid=5846) stop_2 stim started
(worker pid=5846) opt stim started
(worker pid=5821) stop_2 stim started
(worker pid=5821) opt stim started
(worker pid=5821) stop_2 stim started
(worker pid=5821) opt stim started
(worker pid=5811) trial_num 8
(worker pid=5811)   action
(worker pid=5811) 0   left
(worker pid=5811) chosen_action: stop
(worker pid=5811) trial_num 8
(worker pid=5811)   action
(worker pid=5811) 0   left
(worker pid=5811) chosen_action: stop
(worker pid=5817) stop_2 stim started
(worker pid=5817) opt stim started
(worker pid=5841) stop_2 stim started
(worker pid=5841) opt stim started
(worker pid=5817) stop_2 stim started
(worker pid=5817) opt stim started
(worker pid=5841) stop_2 stim started
(worker pid=5841) opt stim started
(worker pid=5856) stop_2 s

(worker pid=5858) gateFRs [30.44444444]
(worker pid=5858) thresholds_crossed [0]
(worker pid=5856) trial_num 9
(worker pid=5856)   action
(worker pid=5856) 0   left
(worker pid=5856) chosen_action: left
(worker pid=5856) trial_num 9
(worker pid=5856)   action
(worker pid=5856) 0   left
(worker pid=5856) chosen_action: left
(worker pid=5812) gateFRs [30.]
(worker pid=5812) thresholds_crossed []
(worker pid=5812) gateFRs [30.]
(worker pid=5812) thresholds_crossed []
(worker pid=5811) gateFRs [27.55555556]
(worker pid=5811) thresholds_crossed []
(worker pid=5811) gateFRs [27.55555556]
(worker pid=5811) thresholds_crossed []
(worker pid=5845) trial_num 9
(worker pid=5845)   action
(worker pid=5845) 0   left
(worker pid=5845) chosen_action: left
(worker pid=5845) trial_num 9
(worker pid=5845)   action
(worker pid=5845) 0   left
(worker pid=5845) chosen_action: left
(worker pid=5814) stop_2 stim started
(worker pid=5814) opt stim started
(worker pid=5814) stop_2 stim started
(worker pid=5814

...

(worker pid=5858) stop_2 stim started
(worker pid=5858) opt stim started
(worker pid=5858) stop_2 stim started
(worker pid=5858) opt stim started
(worker pid=5846) trial_num 12
(worker pid=5846)   action
(worker pid=5846) 0   left
(worker pid=5846) chosen_action: stop
(worker pid=5846) trial_num 12
(worker pid=5846)   action
(worker pid=5846) 0   left
(worker pid=5846) chosen_action: stop
(worker pid=5858) gateFRs [30.22222222]
(worker pid=5858) thresholds_crossed [0]
(worker pid=5858) gateFRs [30.22222222]
(worker pid=5858) thresholds_crossed [0]
(worker pid=5858) trial_num 11
(worker pid=5858)   action
(worker pid=5858) 0   left
(worker pid=5858) chosen_action: left
(worker pid=5858) trial_num 11
(worker pid=5858)   action
(worker pid=5858) 0   left
(worker pid=5858) chosen_action: left
(worker pid=5814) stop_2 stim started
(worker pid=5814) opt stim started
(worker pid=5814) stop_2 stim started
(worker pid=5814) opt stim started
(worker pid=5846) stop_2 stim started
(worker pid=5846

(worker pid=5858) trial_num 14
(worker pid=5858)   action
(worker pid=5858) 0   left
(worker pid=5858) chosen_action: left
(worker pid=5846) gateFRs [27.33333333]
(worker pid=5846) thresholds_crossed []
(worker pid=5858) trial_num 14
(worker pid=5858)   action
(worker pid=5858) 0   left
(worker pid=5858) chosen_action: left
(worker pid=5846) gateFRs [27.33333333]
(worker pid=5846) thresholds_crossed []
(worker pid=5814) trial_num 15
(worker pid=5814)   action
(worker pid=5814) 0   left
(worker pid=5814) chosen_action: stop
(worker pid=5814) trial_num 15
(worker pid=5814)   action
(worker pid=5814) 0   left
(worker pid=5814) chosen_action: stop
(worker pid=5846) trial_num 16
(worker pid=5846)   action
(worker pid=5846) 0   left
(worker pid=5846) chosen_action: stop
(worker pid=5846) trial_num 16
(worker pid=5846)   action
(worker pid=5846) 0   left
(worker pid=5846) chosen_action: stop
(worker pid=5858) stop_2 stim started
(worker pid=5858) opt stim started
(worker pid=5858) stop_2 stim

(worker pid=5814) stop_2 stim started
(worker pid=5814) opt stim started
(worker pid=5814) stop_2 stim started
(worker pid=5814) opt stim started
(worker pid=5846) stop_2 stim started
(worker pid=5846) opt stim started
(worker pid=5846) stop_2 stim started
(worker pid=5846) opt stim started
(worker pid=5858) gateFRs [22.66666667]
(worker pid=5858) thresholds_crossed []
(worker pid=5858) gateFRs [22.66666667]
(worker pid=5858) thresholds_crossed []
(worker pid=5814) gateFRs [26.]
(worker pid=5814) thresholds_crossed []
(worker pid=5814) gateFRs [26.]
(worker pid=5814) thresholds_crossed []
(worker pid=5846) gateFRs [26.22222222]
(worker pid=5846) thresholds_crossed []
(worker pid=5846) gateFRs [26.22222222]
(worker pid=5846) thresholds_crossed []
(worker pid=5858) trial_num 18
(worker pid=5858)   action
(worker pid=5858) 0   left
(worker pid=5858) chosen_action: stop
(worker pid=5858) trial_num 18
(worker pid=5858)   action
(worker pid=5858) 0   left
(worker pid=5858) chosen_action: sto

(worker pid=5846) trial_num 23
(worker pid=5846)   action
(worker pid=5846) 0   left
(worker pid=5846) chosen_action: stop
(worker pid=5846) trial_num 23
(worker pid=5846)   action
(worker pid=5846) 0   left
(worker pid=5846) chosen_action: stop
(worker pid=5814) trial_num 22
(worker pid=5814)   action
(worker pid=5814) 0   left
(worker pid=5814) chosen_action: stop
(worker pid=5814) trial_num 22
(worker pid=5814)   action
(worker pid=5814) 0   left
(worker pid=5814) chosen_action: stop
(worker pid=5846) stop_2 stim started
(worker pid=5846) opt stim started
(worker pid=5846) stop_2 stim started
(worker pid=5846) opt stim started
(worker pid=5814) stop_2 stim started
(worker pid=5814) opt stim started
(worker pid=5814) stop_2 stim started
(worker pid=5814) opt stim started
(worker pid=5858) stop_2 stim started
(worker pid=5858) opt stim started
(worker pid=5858) stop_2 stim started
(worker pid=5858) opt stim started
(worker pid=5846) gateFRs [28.66666667]
(worker pid=5846) thresholds_c

(worker pid=5846) trial_num 27
(worker pid=5846)   action
(worker pid=5846) 0   left
(worker pid=5846) chosen_action: stop
(worker pid=5846) trial_num 27
(worker pid=5846)   action
(worker pid=5846) 0   left
(worker pid=5846) chosen_action: stop
(worker pid=5858) stop_2 stim started
(worker pid=5858) opt stim started
(worker pid=5858) stop_2 stim started
(worker pid=5858) opt stim started
(worker pid=5814) trial_num 26
(worker pid=5814)   action
(worker pid=5814) 0   left
(worker pid=5814) chosen_action: left
(worker pid=5814) trial_num 26
(worker pid=5814)   action
(worker pid=5814) 0   left
(worker pid=5814) chosen_action: left
(worker pid=5858) gateFRs [25.55555556]
(worker pid=5858) thresholds_crossed []
(worker pid=5858) gateFRs [25.55555556]
(worker pid=5858) thresholds_crossed []
(worker pid=5858) trial_num 25
(worker pid=5858)   action
(worker pid=5858) 0   left
(worker pid=5858) chosen_action: stop
(worker pid=5858) trial_num 25
(worker pid=5858)   action
(worker pid=5858) 0  